In [208]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [209]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [210]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
0,Date:Time,Price(USD/MWh),None
1,Min,,
2,Max,,
3,Average,,


In [211]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/04/01) 04-05,39.06,9.4
6,(2017/04/01) 05-06,39.06,9.4
7,(2017/04/01) 06-07,39.06,9.4
8,(2017/04/01) 07-08,67.32,0.0
9,(2017/04/01) 08-09,67.32,0.0
...,...,...,...
740,(2023/05/31) 19-20,125.00,0.0
741,(2023/05/31) 20-21,75.00,0.0
742,(2023/05/31) 21-22,75.00,0.0
743,(2023/05/31) 22-23,56.00,0.0


In [212]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/04/01) 04-05,39.06,9.4,2017-04-01,4
6,(2017/04/01) 05-06,39.06,9.4,2017-04-01,5
7,(2017/04/01) 06-07,39.06,9.4,2017-04-01,6
8,(2017/04/01) 07-08,67.32,0.0,2017-04-01,7
9,(2017/04/01) 08-09,67.32,0.0,2017-04-01,8
...,...,...,...,...,...
740,(2023/05/31) 19-20,125.00,0.0,2023-05-31,19
741,(2023/05/31) 20-21,75.00,0.0,2023-05-31,20
742,(2023/05/31) 21-22,75.00,0.0,2023-05-31,21
743,(2023/05/31) 22-23,56.00,0.0,2023-05-31,22


In [213]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,39.06,9.4,2017-04-01,4
6,39.06,9.4,2017-04-01,5
7,39.06,9.4,2017-04-01,6
8,67.32,0.0,2017-04-01,7
9,67.32,0.0,2017-04-01,8
...,...,...,...,...
740,125.00,0.0,2023-05-31,19
741,75.00,0.0,2023-05-31,20
742,75.00,0.0,2023-05-31,21
743,56.00,0.0,2023-05-31,22


In [214]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5
2017-04-01,39.06,9.4,5,2017,4,1,5
2017-04-01,39.06,9.4,6,2017,4,1,5
2017-04-01,67.32,0.0,7,2017,4,1,5
2017-04-01,67.32,0.0,8,2017,4,1,5
...,...,...,...,...,...,...,...
2023-05-31,125.00,0.0,19,2023,5,31,2
2023-05-31,75.00,0.0,20,2023,5,31,2
2023-05-31,75.00,0.0,21,2023,5,31,2


In [215]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2023 5     27      0.0     0.000              NaN      75.00      56.00   
           28      0.0     0.000              NaN      56.00      56.00   
           29      0.0     0.000              NaN     125.00      56.00   
           30      0.0     0.000              NaN     125.00      56.00   
           31      0.0     0.000              NaN     125.00      56.00   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 4     1     48.951000        9.4        0.0    6.110000 2017-04-01  
           2     39.060000        9.4        9.4    9.400000 2017-04-02  
           3     61.668000        9.4        0.0    1.880000 2017-04-03  
           4     61.668000        9.4        0.0    1.880000 2017-04-04  
           5     57.900000        9.4        0.0    3.133333 2017-04-05  
...                    ...        ...        ...         ...        ...  
2023 5     27    61.541667        0.0        0.0    0.000000 2023-05-27  
           28    56.000000        0.0        0.0    0.000000 2023-05-28  
           29    83.700000        0.0        0.0    0.000000 2023-05-29  
           30    83.700000        0.0        0.0    0.000000 2023-05-30  
           31    83.700000        0.0        0.0    0.000000 2023-05-31  

[2118 rows x 10 columns]

In [216]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,5,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,6,2017,4,1,5,367.164,off
2017-04-01,67.32,0.0,7,2017,4,1,5,0.000,standard
2017-04-01,67.32,0.0,8,2017,4,1,5,0.000,standard
...,...,...,...,...,...,...,...,...,...
2023-05-31,125.00,0.0,19,2023,5,31,2,0.000,standard
2023-05-31,75.00,0.0,20,2023,5,31,2,0.000,standard
2023-05-31,75.00,0.0,21,2023,5,31,2,0.000,standard


In [217]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2023 5     27      0.0     0.000              NaN      75.00      56.00   
           28      0.0     0.000              NaN      56.00      56.00   
           29      0.0     0.000              NaN     125.00      56.00   
           30      0.0     0.000              NaN     125.00      56.00   
           31      0.0     0.000              NaN     125.00      56.00   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 4     1     48.951000        9.4        0.0    6.110000 2017-04-01  ...   
           2     39.060000        9.4        9.4    9.400000 2017-04-02  ...   
           3     61.668000        9.4        0.0    1.880000 2017-04-03  ...   
           4     61.668000        9.4        0.0    1.880000 2017-04-04  ...   
           5     57.900000        9.4        0.0    3.133333 2017-04-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2023 5     27    61.541667        0.0        0.0    0.000000 2023-05-27  ...   
           28    56.000000        0.0        0.0    0.000000 2023-05-28  ...   
           29    83.700000        0.0        0.0    0.000000 2023-05-29  ...   
           30    83.700000        0.0        0.0    0.000000 2023-05-30  ...   
           31    83.700000        0.0        0.0    0.000000 2023-05-31  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 4     1                   NaN                 NaN       122.2   
           2                   NaN                 NaN       225.6   
           3                   NaN                 NaN        37.6   
           4                   NaN                 NaN        37.6   
           5                   NaN                 NaN        75.2   
...                            ...                 ...         ...   
2023 5     27                  NaN                 NaN         0.0   
           28                  NaN                 NaN         0.0   
           29                  NaN                 NaN         0.0   
           30                  NaN                 NaN         0.0   
           31                  NaN                 NaN         0.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 4     1                0.0             NaN             NaN   4773.132   
           2                NaN             NaN             NaN   8811.936   
           3                0.0             NaN             NaN   1468.656   
           4                0.0             NaN             NaN   1468.656   
           5                0.0             NaN             NaN   2937.312   
...                         ...             ...             ...        ...   
2023 5     27               0.0             NaN             NaN      0.000   
           28               NaN             NaN             NaN      0.000   
           29               0.0             NaN             NaN      0.000   
           30               0.0             NaN             NaN      0.000   
           31               0.0             NaN             NaN      0.000   

                standard_value  morning_value  evening_value  
year month

In [218]:
daily.to_csv(out + pref + '_daily.csv')

In [219]:
hourly.to_csv(out + pref + '_hourly.csv')

In [220]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-04-01,4,39.06,9.4,off
2017-04-01,5,39.06,9.4,off
2017-04-01,6,39.06,9.4,off
2017-04-01,7,67.32,0.0,standard
2017-04-01,8,67.32,0.0,standard
...,...,...,...,...
2023-05-31,19,125.00,0.0,standard
2023-05-31,20,75.00,0.0,standard
2023-05-31,21,75.00,0.0,standard


In [221]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [222]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.668,39.06,53.606050,9.4,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.00000,NaN
1,2017,5,107.096774,3970.077419,57.678,37.07,50.891763,10.0,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.40404,NaN
2,2017,6,26.400000,1176.384000,68.368,64.40,66.384000,4.4,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.00000,NaN
3,2017,7,273.333333,11540.133333,68.404,42.22,54.766500,20.0,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.00000,NaN
4,2017,8,380.833333,22038.583333,75.400,33.40,62.785417,20.0,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2023,1,0.000000,0.000000,81.650,62.00,75.058065,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
69,2023,2,0.000000,0.000000,81.650,62.00,75.170536,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
70,2023,3,0.000000,0.000000,81.650,62.00,75.797581,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
71,2023,4,0.000000,0.000000,81.650,62.00,74.560833,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN


In [223]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.668,39.06,53.606050,9.4,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.00000,NaN
1,2017,5,107.096774,3970.077419,57.678,37.07,50.891763,10.0,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.40404,NaN
2,2017,6,26.400000,1176.384000,68.368,64.40,66.384000,4.4,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.00000,NaN
3,2017,7,273.333333,11540.133333,68.404,42.22,54.766500,20.0,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.00000,NaN
4,2017,8,380.833333,22038.583333,75.400,33.40,62.785417,20.0,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2023,1,0.000000,0.000000,81.650,62.00,75.058065,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
69,2023,2,0.000000,0.000000,81.650,62.00,75.170536,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
70,2023,3,0.000000,0.000000,81.650,62.00,75.797581,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN
71,2023,4,0.000000,0.000000,81.650,62.00,74.560833,0.0,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.00000,NaN


In [224]:
monthly=monthly.set_index(['year','month'])

In [225]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [226]:
monthly.to_csv(out + pref + '_monthly.csv')

In [227]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [228]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,6404.075269,43.387500,93.420376,43.250000,71.781300,13.333333,2.365591,6.401434,43.250000,...,0.0,1.985263e+05,327.000000,385.000000,0.0,0.0,13.333333,0.476009,NaN,NaN
1,2,6215.296473,42.977699,86.089231,43.096095,69.228553,13.488757,1.910059,6.255878,43.096095,...,0.0,1.750642e+05,306.166667,349.166667,0.0,0.0,13.450000,0.000000,NaN,NaN
2,3,4390.984146,42.748169,88.355061,46.293415,68.716177,8.658537,1.707317,4.427846,46.293415,...,0.0,1.200202e+05,297.666667,329.000000,0.0,0.0,10.000000,0.137812,NaN,NaN
3,4,6524.029848,43.754000,78.004000,43.667143,62.602740,12.771429,2.509524,6.348484,43.667143,...,0.0,1.957209e+05,322.714286,369.285714,0.0,0.0,12.771429,0.853230,NaN,NaN
4,5,3322.595392,34.730000,85.546406,41.481429,64.724698,8.571429,1.566820,4.162058,41.481429,...,0.0,1.030005e+05,325.428571,386.571429,0.0,0.0,8.571429,0.292568,NaN,NaN
5,6,10044.841895,55.036227,110.299671,37.527763,76.990835,17.294737,2.105263,7.961711,37.527763,...,0.0,2.544693e+05,267.666667,316.333333,0.0,0.0,14.066667,1.447368,NaN,NaN
6,7,48466.706582,65.766733,129.198481,43.864177,91.495701,56.629114,9.367089,30.307954,43.864177,...,0.0,1.276290e+06,278.500000,325.500000,0.0,0.0,40.833333,15.400000,NaN,NaN
7,8,6330.061491,45.911542,110.970870,44.024720,83.186013,12.298137,2.329193,5.953675,44.024720,...,0.0,1.698566e+05,281.833333,333.500000,0.0,0.0,13.333333,2.500000,NaN,NaN
8,9,5956.745556,33.786000,76.404611,35.813333,61.296218,16.666667,2.333333,7.618519,35.813333,...,0.0,1.787024e+05,320.333333,371.666667,0.0,0.0,16.666667,0.000000,NaN,NaN
9,10,7758.065591,35.226000,79.273763,34.011667,62.412353,20.000000,3.010753,9.322581,34.011667,...,0.0,2.405000e+05,328.500000,383.500000,0.0,0.0,20.000000,0.000000,NaN,NaN


In [229]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [230]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume       value
month hour                                   
1     0      43.250000  13.333333  578.500000
      1      43.250000  13.333333  578.500000
      2      43.250000  13.333333  578.500000
      3      43.250000  13.333333  578.500000
      4      43.250000  13.333333  578.500000
...                ...        ...         ...
12    19    102.688258   4.861935  250.741987
      20     89.184645   6.778065  335.670761
      21     89.184645   6.778065  335.670761
      22     51.652000  18.000000  853.200000
      23     51.652000  18.000000  853.200000

[288 rows x 3 columns]

In [231]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [232]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price  volume    value
year month hour                             
2017 4     0      39.060000     9.4  367.164
           1      39.060000     9.4  367.164
           2      39.060000     9.4  367.164
           3      39.060000     9.4  367.164
           4      39.060000     9.4  367.164
...                     ...     ...      ...
2023 5     19    107.419355     0.0    0.000
           20     69.483871     0.0    0.000
           21     69.483871     0.0    0.000
           22     56.000000     0.0    0.000
           23     56.000000     0.0    0.000

[1752 rows x 3 columns]

In [233]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [234]:
from statsmodels.tsa.stattools import pacf

In [235]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.0,1.0,1.000000
1,0.574299,NaN,NaN,0.741971
2,-0.145788,NaN,NaN,0.053974
3,0.092269,NaN,NaN,0.048295
4,0.375340,NaN,NaN,0.184696
5,0.212745,NaN,NaN,-0.192539
6,0.057490,NaN,NaN,-0.211158
7,0.012123,NaN,NaN,0.346759
8,0.058712,NaN,NaN,-0.046780
9,-0.025035,NaN,NaN,-0.106274


In [236]:
pacfs.to_csv(out + pref + '_pacfs.csv')